# Install libraries

In [ ]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 65.5 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata #if token is stored in secrets

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')


In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(repo_id = hf_model)

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #here you might have stored your books
import os
from langchain.document_loaders import TextLoader

Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/MyDrive/books/' #get your books from here
documents = []
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        loader = TextLoader(file_path)
        docs = loader.load()
        documents.extend(docs)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

#Initialize the text splitter with your desired chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)

#Split the documents into smaller chunks
docs = text_splitter.split_documents(documents)

#Initialize the embeddings model (for example, sentence-transformers)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-MiniLM-L-6-v3")

#Create the FAISS index from the chunked documents
vector_db = FAISS.from_documents(docs, embeddings)

<ipython-input-6-74c7bb2171d9>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-MiniLM-L-6-v3")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vector_db.save_local("/content/faiss_index")

In [ ]:
test_text = "Whhat is a bug"
query_result = embeddings.embed_query(test_text)
query_result

[-0.22928817570209503,
 0.4503999948501587,
 -0.3992973566055298,
 0.20103074610233307,
 0.12912209331989288,
 -0.6786773204803467,
 0.44262343645095825,
 -0.3670101761817932,
 0.098402239382267,
 0.5739308595657349,
 -0.5539441108703613,
 -0.6732749938964844,
 0.47278058528900146,
 -0.3004656434059143,
 0.22636640071868896,
 0.2579929828643799,
 0.4210004508495331,
 0.07580199092626572,
 -0.3921644985675812,
 0.03899717330932617,
 -0.6292672157287598,
 0.7622119188308716,
 -0.005722310394048691,
 0.029538925737142563,
 -0.20042875409126282,
 0.020026136189699173,
 -0.16316044330596924,
 0.03451048955321312,
 0.34400129318237305,
 0.032210513949394226,
 0.5145990252494812,
 0.4386042654514313,
 0.29864075779914856,
 0.17789137363433838,
 -0.6229569911956787,
 0.13611814379692078,
 0.4870556890964508,
 -0.0795946791768074,
 -0.24379637837409973,
 0.19763271510601044,
 -0.21484549343585968,
 0.7405107021331787,
 -0.16167126595973969,
 -0.5391650795936584,
 -0.5173384547233582,
 -0.195208

# Debuggin - check the dimensions

In [ ]:
characters = len(test_text)
dimensions = len(query_result)
print(f"The {characters} character sentence was transformed into a {dimensions} dimension vector")

The 14 character sentence was transformed into a 384 dimension vector


In [ ]:
#Similarity search with FAISS
query = "What is ethical hacking?"
results = vector_db.similarity_search(query, k=2)

for result in results:
    print(result)

page_content='https://creativecommons.org/licenses/by-nc-nd/4.0/



218  Ethical Hacking

 communications. Journal of Cybersecurity, 1(1), 69–79, https://doi.org/10.1093/
cybsec/tyv009. 

See also: CRYPTO WARS, MANDATORY DECRYPTION, PUBLIC-
KEY INFRASTRUCTURE

Ethical Hacking

Ethical hacking is the practice of using the techniques of malicious hackers 
to test the security of  computer systems, by exposing vulnerabilities. 
Ethical hackers usually have the permission of the system managers, but 
even if  not, they will inform managers, rather than making the vulnerabil-
ity public or exploiting it for gain.

Further reading:
Palmer, C.C., 2001. Ethical hacking. IBM Systems Journal, 40(3), 769–80, https://

doi.org/10.1147/sj.403.0769.' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/final/Dictionary of Privacy, Data Protection and Information Security.txt'}
page_content='The way the term ‘ethical hacking’ is used appears to presuppose a clear and 
unilateral solution to th

# Test the model

In [ ]:
from langchain.prompts.prompt import PromptTemplate

input_template = """Answer the question based only on the following context. Keep your answers short and succinct.

Context to answer question:
{context}

Question to be answered: {question}
Response:"""


prompt = PromptTemplate(template=input_template,
                        input_variables=["context", "question"])

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_db.as_retriever(search_kwargs={"k": 2}), #top 2 results only, speed things up
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt},
)

In [ ]:
answer = qa_chain.invoke("What is a bug?")

answer

{'query': 'What is a bug?',
 'result': ' A bug is an error in the code of a computer program.',
 'source_documents': [Document(id='c2b91836-3f38-4336-b08d-c245003fbcb8', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/final/Dictionary of Privacy, Data Protection and Information Security.txt'}, page_content='https://creativecommons.org/licenses/by-nc-nd/4.0/\n\n\n\nBusiness Case  6 3\n\nBug\n\nA bug is an error in the code of a computer program. Bugs can have differ-\nent kinds of impact, from zero to minor business efficiency consequences \nto catastrophic security and system integrity issues.'),
  Document(id='87fecef6-97a2-4fe4-be65-0400e265322c', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/final/Application_of_Machine_Learning_and_Data_Mining.txt'}, page_content='24. Li, Z.; Tan, L.; Wang, X.; Lu, S.; Zhou, Y.; Zhai, C. Have things changed now? An empirical study of bug characteristics in\nmodern open-source software. In Proceedings of the 1st Workshop on

# Test if the model is using sources

In [ ]:
answer.keys()
answer['source_documents']
answer['source_documents'][0].page_content
print(answer['source_documents'][0].page_content)

https://creativecommons.org/licenses/by-nc-nd/4.0/



Business Case  6 3

Bug

A bug is an error in the code of a computer program. Bugs can have differ-
ent kinds of impact, from zero to minor business efficiency consequences 
to catastrophic security and system integrity issues.


In [ ]:
answer['source_documents'][0].metadata

{'source': '/content/drive/MyDrive/Colab Notebooks/final/Dictionary of Privacy, Data Protection and Information Security.txt'}